# Summary
Goal: Given various medical indicators, predict the health outcomes of horses.




























## What I know so far...
**Iteration 1.**
Blindly adding everything together.
Removed Id.
Using RandomForest to start.































## Scoring Versions
    Iteration 1. Baseline Model
    model_base 1.0 1.0 1.0
    model_base 0.7206477732793523 0.7247937820693239 0.7197463527611024
    model_tune 0.8876518218623481 0.8913947148261604 0.8878737029137665
    model_tune 0.708502024291498 0.7146934707306224 0.7081103777851071
    

In [25]:
import pandas as pd
import numpy as np
import seaborn as sns
pd.options.display.max_columns = 200

In [26]:
df = pd.read_csv('input/train.csv')
samp = pd.read_csv('input/test.csv')
# df = pd.read_csv('/kaggle/input/playground-series-s3e22/train.csv')
# samp = pd.read_csv('/kaggle/input/playground-series-s3e22/test.csv')

In [27]:
num = [x for x in df.columns if df.dtypes[x] in ('float', 'int64')]
cat = [x for x in df.columns if df.dtypes[x] == 'object']
target = 'outcome'
cat.remove(target)
num.remove('id')

In [28]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=.2, random_state=0)

# Explore

In [29]:
train.isnull().any()

id                       False
surgery                  False
age                      False
hospital_number          False
rectal_temp              False
pulse                    False
respiratory_rate         False
temp_of_extremities      False
peripheral_pulse         False
mucous_membrane          False
capillary_refill_time    False
pain                     False
peristalsis              False
abdominal_distention     False
nasogastric_tube         False
nasogastric_reflux       False
nasogastric_reflux_ph    False
rectal_exam_feces        False
abdomen                  False
packed_cell_volume       False
total_protein            False
abdomo_appearance        False
abdomo_protein           False
surgical_lesion          False
lesion_1                 False
lesion_2                 False
lesion_3                 False
cp_data                  False
outcome                  False
dtype: bool

In [30]:
train[target].value_counts()

lived         458
died          321
euthanized    209
Name: outcome, dtype: int64

# Pipeline

In [31]:
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import OneHotEncoder, StandardScaler, PolynomialFeatures
from sklearn.impute import SimpleImputer

In [32]:
class DataSelect(BaseEstimator, TransformerMixin):
    def __init__(self, attributes):
        self.attributes = attributes
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        return X[self.attributes]

In [33]:
num_pp = Pipeline([
    ('dataselect', DataSelect(num))
    , ('impute', SimpleImputer(strategy='median'))
    , ('scaler', StandardScaler())
])

cat_pp = Pipeline([
    ('dataselect', DataSelect(cat))
    , ('encode', OneHotEncoder(sparse_output=False, handle_unknown='ignore'))
])

pipe = FeatureUnion([
    ('numeric', num_pp)
    , ('categorical', cat_pp)
])

pipe.fit(train)
train_pre = pipe.transform(train)
test_pre = pipe.transform(test)

# Base Model

In [34]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [35]:
lin_cls = LogisticRegression(max_iter=1000)
lin_cls.fit(train_pre, train[target])
cross_val_score(lin_cls, train_pre, train[target]).mean()

0.6619391888427422

In [36]:
svc_cls = LinearSVC(dual=True, max_iter=8000)
svc_cls.fit(train_pre, train[target])
cross_val_score(svc_cls, train_pre, train[target]).mean()

0.6507921858175665

In [37]:
dec_cls = DecisionTreeClassifier(random_state=0)
dec_cls.fit(train_pre, train[target])
cross_val_score(dec_cls, train_pre, train[target]).mean()

0.5677690611700764

In [38]:
ran_cls = RandomForestClassifier(random_state=0)
ran_cls.fit(train_pre, train[target])
cross_val_score(ran_cls, train_pre, train[target]).mean()

0.6932574475721685

In [39]:
model_base = RandomForestClassifier(random_state=0)
model_base.fit(train_pre, train[target])
model_base.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 0,
 'verbose': 0,
 'warm_start': False}

# Tune Model

In [40]:
# from sklearn.model_selection import GridSearchCV
# params = {
#     'random_state': [0]
#     , 'n_estimators': [100, 300, 600]
#     , 'max_depth': [2, 4, 8]
# }
# grid = GridSearchCV(model_base, param_grid=params)
# grid.fit(train_pre, train[target])
# grid.best_params_

In [41]:
model_tune = RandomForestClassifier(random_state=0, n_estimators=100, max_depth=8)

In [42]:
model_tune.fit(train_pre, train[target]);

# Validation

In [43]:
from sklearn.metrics import accuracy_score, precision_score, f1_score, roc_auc_score

In [44]:
print('model_base'
, accuracy_score(train[target], model_base.predict(train_pre))
, precision_score(train[target], model_base.predict(train_pre), average='weighted')
, f1_score(train[target], model_base.predict(train_pre), average='weighted')
)

print('model_base'
, accuracy_score(test[target], model_base.predict(test_pre))
, precision_score(test[target], model_base.predict(test_pre), average='weighted')
, f1_score(test[target], model_base.predict(test_pre), average='weighted')
)

print('model_tune'
, accuracy_score(train[target], model_tune.predict(train_pre))
, precision_score(train[target], model_tune.predict(train_pre), average='weighted')
, f1_score(train[target], model_tune.predict(train_pre), average='weighted')
)

print('model_tune'
, accuracy_score(test[target], model_tune.predict(test_pre))
, precision_score(test[target], model_tune.predict(test_pre), average='weighted')
, f1_score(test[target], model_tune.predict(test_pre), average='weighted')
)

model_base 1.0 1.0 1.0
model_base 0.7206477732793523 0.7247937820693239 0.7197463527611024
model_tune 0.8876518218623481 0.8913947148261604 0.8878737029137665
model_tune 0.708502024291498 0.7146934707306224 0.7081103777851071


# Submission

In [45]:
pd.DataFrame({
    'id': samp['id']
    , 'outcome': model_tune.predict(pipe.transform(samp))
}).to_csv('submission.csv', index=False)